In [1]:
import numpy as np
import cmath
from scipy.optimize import curve_fit
import tensorflow as tf

In [2]:
# 下载数据集
minist = tf.keras.datasets.mnist
(train_x, train_y), (test_x, test_y) = minist.load_data()

In [3]:
# 输出数据相关信息
print("Training set:", len(train_x))
print("Testing set:", len(test_x))

Training set: 60000
Testing set: 10000


In [4]:
print("train_x", train_x.shape, train_x.dtype)
print("train_y", train_y.shape, train_y.dtype)
shape0, shape1, shape2 = train_x.shape

train_x (60000, 28, 28) uint8
train_y (60000,) uint8


In [5]:
new_train_x = np.zeros((shape0, shape1 * shape2))
for i in range(shape0):
    new_train_x[i] = train_x[i].flatten()
print(new_train_x.shape)

(60000, 784)


Given data $(x_1,y_1), \cdots, (x_n,y_n)$ from $\mathbb{R}^d \times \mathbb{R}$, we find the predictor $h_{n,N} \in \mathcal{H}_N$ via ERM with squared loss.

In [10]:
class RFF:
    __d = 0
    __N = 0
    
    def __init__(self, d: int, N: int) -> None:
        self.__d = d
        self.__N = N

    @classmethod
    def fit(self, x: np.ndarray, y: np.ndarray):
        a = np.zeros(self.__N)
        return curve_fit(self.__h, x, y, a)

    @classmethod
    def __phi(self, x: np.ndarray, v: np.ndarray) -> any:
        '''
        The kernel function

        Input:
        - x: The input data
        - v: The auxiliary matrix

        Returns:
        - The result of the function
        '''

        return np.exp(cmath.sqrt(-1) * np.dot(v, x))
    
    @classmethod
    def __h(self, x: np.ndarray, a: np.ndarray) -> int:
        '''
        Map the input image to its label with Random Fourier Features.

        Inputs:
        - x: The matrix of the image
        - N: The number of features 
        - a: The array of parameters

        Returns:
        - The predicted label
        '''

        v = np.random.randn(self.__N, self.__d)
        result = 0
        for i in range(1, self.__N):
            result += (a[i] * self.__phi(x, v[i]))
        return result

In [8]:
d = train_x.shape[1] * train_x.shape[2] # 数据维度
maxN = 10 # 设置最大特征数

In [7]:
# 定义核函数
def phi(x: np.ndarray, v: np.ndarray) -> any:
    '''
    The kernel function

    Input:
    - x: The input data
    - v: The auxiliary matrix

    Returns:
    - The result of the function
    '''

    return np.exp(cmath.sqrt(-1) * np.dot(v, x))

In [ ]:
# 定义傅里叶函数
def h(x: np.ndarray, N: int, a: np.ndarray) -> int:
    '''
    Map the input image to its label with Random Fourier Features.

    Inputs:
    - x: The matrix of the image
    - N: The number of features 
    - a: The array of parameters

    Returns:
    - The predicted label
    '''

    v = np.random.randn(N, d)


In [12]:
print(train_y)

[5 0 4 ... 5 6 8]


In [11]:
rff = RFF(d, 10)
result = rff.fit(new_train_x, train_y)

TypeError: object of type 'numpy.float64' has no len()